In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [3]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FieldGoalsMade3&agg=PerGame"

In [5]:
for year in years:
    with open("maded_threepointers_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [9]:
dfs = []
for year in years:
    with open("maded_threepointers_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    threepointers_data   = pd.read_html(str(player_table))[0]
    threepointers_data["Year"] = year
    dfs.append(threepointers_data)

In [10]:
threepointers_data = pd.concat(dfs)
threepointers_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"SCEPANOVIC, VLADOSCEPANOVIC, V.",PARPartizanPartizan,14,6.36,3.21,50.56%,2001
1,2,"HILL, KYLEHILL, K.",ASVAsvel BasketAsvel Basket,14,6.50,2.71,41.76%,2001
2,3,"NICOLA, MARCELONICOLA, M.",TREBenetton BasketBenetton Basket,20,6.60,2.50,37.88%,2001
3,4,"LAKOVIC, JAKALAKOVIC, J.",MESKRKA Novo MestoKRKA Novo Mesto,14,5.57,2.43,43.59%,2001
4,5,"BASILE, GIANLUCABASILE, G.",FORSkipper BolognaSkipper Bologna,17,5.47,2.35,43.01%,2001
...,...,...,...,...,...,...,...,...
45,46,"PAPANIKOLAOU, KOSTASPAPANIKOLAOU, K.",OLYOlympiacos PiraeusOlympiacos Piraeus,37,3.76,1.24,33.09%,2021
46,47,"DATOME, LUIGIDATOME, L.",AXMAX Armani Exchange MilanAX Armani Exchange ...,26,3.15,1.23,39.02%,2021
47,48,"STRELNIEKS, JANISSTRELNIEKS, J.",ZALZalgiris KaunasZalgiris Kaunas,23,3.61,1.22,33.73%,2021
48,49,"WILLIAMS, DERRICKWILLIAMS, D.",MTAMaccabi Playtika Tel AvivMaccabi Playtika T...,35,3.17,1.20,37.84%,2021


In [11]:
engine = create_engine('sqlite:///maded_threepointers_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
threepointers_data.to_sql(
    name = "maded_threepointers_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [15]:
maded_threepointers_leaders_select = session.execute('select * from "maded_threepointers_leaders"')
maded_threepointers_leaders_select.all()

[(0, 1, 'SCEPANOVIC, VLADOSCEPANOVIC, V.', 'PARPartizanPartizan', 14, 6.36, 3.21, '50.56%', 2001),
 (1, 2, 'HILL, KYLEHILL, K.', 'ASVAsvel BasketAsvel Basket', 14, 6.5, 2.71, '41.76%', 2001),
 (2, 3, 'NICOLA, MARCELONICOLA, M.', 'TREBenetton BasketBenetton Basket', 20, 6.6, 2.5, '37.88%', 2001),
 (3, 4, 'LAKOVIC, JAKALAKOVIC, J.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 5.57, 2.43, '43.59%', 2001),
 (4, 5, 'BASILE, GIANLUCABASILE, G.', 'FORSkipper BolognaSkipper Bologna', 17, 5.47, 2.35, '43.01%', 2001),
 (5, 6, 'BROWN, MARCUSBROWN, M.', 'EFSEfes PilsenEfes Pilsen', 19, 5.47, 2.32, '42.31%', 2001),
 (6, 7, 'ROLLER, PASCALROLLER, P.', 'FRAOpel SkylinersOpel Skyliners', 14, 5.86, 2.29, '39.02%', 2001),
 (7, 8, 'HOLDEN, JRHOLDEN, JR', 'AEKAEK AthensAEK Athens', 20, 7.45, 2.25, '30.20%', 2001),
 (8, 8, 'BOOKER, MELVINBOOKER, M.', 'PESScavolini PesaroScavolini Pesaro', 20, 5.8, 2.25, '38.79%', 2001),
 (9, 10, 'SLANINA, DONATASSLANINA, D.', 'ZALZalgirisZalgiris', 14, 6.86, 2.21, '32.29%', 2